In [2]:
%load_ext autoreload
%autoreload 2

import os
import zarr
import dask.array as da
import napari
import sys
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from ete3 import Tree

sys.path.append('../libraries')
import family_graph as fg
from cells_database import Base, CellDB, TrackDB

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
os.environ['NAPARI_PERFMON'] = '0'

In [4]:
labels_zarr_path = r'D:\kasia\tracking\E6_exp\code\tests\example_track_labels.zarr'
labels = da.from_zarr(labels_zarr_path)

In [5]:
# display the labels
viewer = napari.Viewer()
labels_layer = viewer.add_labels(labels,name='Labels')
fg.viewer = viewer

In [6]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20_dev\cells_database_2tables.db'
engine = create_engine(f'sqlite:///{new_db_path}')

# send engine to the library
fg.engine = engine

In [7]:
# create a plot widget
t_max = viewer.dims.range[0][1]
plot_widget = fg.build_lineage_widget(t_max)
fg.plot_widget = plot_widget


# add lineage graph
viewer.window.add_dock_widget(plot_widget,area='bottom',name='family')

In [8]:
# connect lineage graph update
labels_layer.events.selected_label.connect(fg.update_lineage_display)

<function family_graph.update_lineage_display(event)>

### Test

In [9]:
my_root = 1

plot_view = plot_widget.getItem(0,0)
plot_view.clear()

# buid the tree
tree = fg.build_Newick_tree(engine, my_root)

In [10]:
# update the widget with the tree
plot_view = fg.render_tree_view(plot_view,tree,viewer)